In [1]:
import time
import folium
import geocoder

import pandas as pd

startClk = time.time()

def calc_distance(data): # 거리 계산
    distance = []
    
    n = len(data)
    
    error = 0
    
    add = 0
    
    for i in data.index:
        
        dis_lat =  data.loc[i,'위도']
        dis_long = data.loc[i,'경도']
        
        if dis_lat == "ERROR" or dis_long == "ERROR":
            data.loc[i,'거리'] = error
        
        else :
            distance.insert(i,(((anu_lat - dis_lat) ** 2 + (anu_long - dis_long) ** 2) ** 0.5) * 100000)
            data.loc[i,'거리'] = distance[add]
            add = add + 1
    
    
    return data

def create_form(data,form,mini,maxi): # 데이터 프레임 분류하기
    n = len(data)
    
    for i in range(n):
        if data.loc[i,'거리'] > mini and data.loc[i,'거리'] < maxi:
            data.loc[i,'분류'] = form
            
    return data
            
def create_form_data(data,form,result_data): # 분류값으로 데이터 프레임 만들기
    insert_data = 0
    
    for i in data.index:
        if  data.loc[i,'분류'] == form:
            result_data.loc[insert_data,'이름'] = data.loc[i,'이름']
            result_data.loc[insert_data,'위도'] = data.loc[i,'위도']
            result_data.loc[insert_data,'경도'] = data.loc[i,'경도']
            result_data.loc[insert_data,'거리'] = data.loc[i,'거리']
            insert_data = insert_data + 1
            
    return result_data

def heapify(data, index, size):
    # 완전이진트리는 배열 하나로 트리 구현 가능
    largest = index
    left_index = 2 * index + 1
    right_index = 2 * index + 2
    # 왼쪽 자식이 현재 요소보다 크면 인덱스 교체
    if left_index < size and data.loc[left_index,'거리'] > data.loc[largest,'거리']:
        largest = left_index
    # 오른쪽 자식이 현재 요소보다 크면 인덱스 교체
    if right_index < size and data.loc[right_index,'거리'] > data.loc[largest,'거리']:
        largest = right_index
    # 교체된 적이 있다면 교체된 index와 largest 요소값 교체
    if largest != index:
        data.loc[largest,'거리'], data.loc[index,'거리'] = data.loc[index,'거리'], data.loc[largest,'거리']
        # 변경되었으면 변경된 부분을 중심으로 다시 한번 heapify
        heapify(data, largest, size)


def heap_sort(data):
    n = len(data)
    # 최초 힙
    # 트리의 절반부터 거꾸로 올라가며 heapify를 하는 것이 효율적
    for i in range(n // 2 - 1, -1, -1):
        heapify(data, i, n)
    # 한번 구성된 힙을 정렬
    # 가장 큰 값(루트)를 가장 끝 값으로 이동한 후 힙 생성.
    for i in range(n - 1, 0, -1):
        data.loc[0,'거리'], data.loc[i,'거리'] = data.loc[i,'거리'], data.loc[0,'거리']
        heapify(data, 0, i)
    return data
            
def merge_data(orgin_data,add_data):
    start = len(orgin_data)
    end = start + len(add_data)
    i = 0
    
    while start < end:
        orgin_data.loc[start,'이름'] = add_data.loc[i,'이름']
        orgin_data.loc[start,'위도'] = add_data.loc[i,'위도']
        orgin_data.loc[start,'경도'] = add_data.loc[i,'경도']
        orgin_data.loc[start,'거리'] = add_data.loc[i,'거리']
        start = start + 1
        i = i + 1

    return orgin_data

def view_marker(data,num):
    for i in range(num): # 정렬된 가까운 식당 30개 마커 찍기
        sub_title = data.loc[i,'이름']
        sub_lat = data.loc[i,'위도']
        sub_long = data.loc[i,'경도']
        
        folium.Marker([sub_lat,sub_long],tooltip = sub_title).add_to(m)

num = 100000
excel_data = pd.read_excel('restaurant_little.xlsx', header = 0) # 액셀 데이터 프레임 형태로 불러오기

sort_data_near = pd.DataFrame(columns=['이름','위도','경도','거리','분류']) # 정렬할 데이터 프레임 생성
sort_data_average = pd.DataFrame(columns=['이름','위도','경도','거리','분류'])
sort_data_far = pd.DataFrame(columns=['이름','위도','경도','거리','분류'])
sort_data = pd.DataFrame(columns=['이름','위도','경도','거리','분류'])

anu = geocoder.ip('me') # 현재 위도를 구하기 위한 내장 함수 
    
anu_lat = anu.lat # 현재 위도
anu_long = anu.lng # 현재 경도

calc_distance(excel_data) # 거리 계산 

m = folium.Map(location=[anu_lat, anu_long],zoom_start=16) # 현재 위치 기준으로 줌

create_form(excel_data,"near",0,101) # 거리값을 이용하여 분류 값 생성
create_form(excel_data,"average",100,1001)
create_form(excel_data,"far",1000,1000001)

create_form_data(excel_data,"near",sort_data_near) # 특정 분류 값을 가진 데이터 프레임 생성
create_form_data(excel_data,"average",sort_data_average)
create_form_data(excel_data,"far",sort_data_far)

heap_sort(sort_data_near) # 데이터 프레임 정렬
heap_sort(sort_data_average)
heap_sort(sort_data_far)

unmerged_data = merge_data(sort_data_near,sort_data_average) # 데이터 프레임 합치기
merged_data = merge_data(unmerged_data,sort_data_far)

view_marker(merged_data,30)

folium.Marker([anu_lat,anu_long],tooltip = "현재 위치").add_to(m) # 현재 위치 마커 찍기
    
print("\n코드 실행 시간 : ", time.time() - startClk, "초")

m


코드 실행 시간 :  1.6407580375671387 초
